In [1]:
# Installer les dépendances
!pip install streamlit ultralytics pyngrok transformers pillow --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 77.8 MB/s eta 0:00:00


In [2]:
from pyngrok import ngrok

# Remplace TON_AUTHTOKEN_ICI par ton token ngrok
!ngrok authtoken 2vdJFZExAVhUhUeOJZjhTHOLFhG_3noX2p3A3rDQ3KYjAKFgF


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# Installation des dépendances
!pip install --quiet streamlit ultralytics transformers torch pillow pyngrok scikit-learn==1.7.2 xgboost

# Vérifier que le fichier modèle est présent
import os
if not os.path.exists("/content/blender_final_per_label_thresholds (1).pkl"):
    print("⚠️  Fichier modèle non trouvé, l'application fonctionnera en mode démonstration")

# Lancer Streamlit
from pyngrok import ngrok
import threading
import subprocess
import time

def run_streamlit():
    subprocess.run(["streamlit", "run", "streamlit_app.py", "--server.port", "8501", "--server.address", "0.0.0.0"])

# Démarrer dans un thread
thread = threading.Thread(target=run_streamlit)
thread.daemon = True
thread.start()

time.sleep(8)  # Attendre que Streamlit démarre

# Créer le tunnel ngrok
public_url = ngrok.connect(8501)
print(f"🎯 Application accessible à: {public_url}")

🎯 Application accessible à: NgrokTunnel: "https://6708a0f459de.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!ls -lh "blender_final_per_label_thresholds (1).pkl"


-rw-r--r-- 1 root root 403M Oct 17 22:19 'blender_final_per_label_thresholds (1).pkl'


In [5]:
%%writefile streamlit_app.py
import streamlit as st
from ultralytics import YOLO
from PIL import Image, ImageDraw
import pandas as pd
import numpy as np
import joblib
import re
import shap
import matplotlib.pyplot as plt
import io
import base64
import torch
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import cv2
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
#VERSION FINALE
# --- DÉFINITION DE LA CLASSE FIXEDBLENDINGPREDICTOR ---
class FixedBlendingPredictor:
    def __init__(self):
        self.base_models = {}
        self.meta_model = None
        self.feature_names = None
        self.top_drugs = None
        self.top_reactions = None
        self.per_label_thresholds = None

    def create_base_models(self):
        from sklearn.ensemble import RandomForestClassifier
        from sklearn.linear_model import LogisticRegression
        from sklearn.multiclass import OneVsRestClassifier
        import xgboost as xgb

        self.base_models = {
            'xgb1': OneVsRestClassifier(xgb.XGBClassifier(n_estimators=100, max_depth=8, learning_rate=0.1, random_state=42), n_jobs=-1),
            'rf1': OneVsRestClassifier(RandomForestClassifier(n_estimators=100, max_depth=15, random_state=44, n_jobs=-1), n_jobs=-1),
            'lr1': OneVsRestClassifier(LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=1000, random_state=46), n_jobs=-1),
        }

    def train_base_models(self, X_train, y_train):
        from tqdm import tqdm
        for name, model in tqdm(self.base_models.items(), desc='Training base models'):
            try:
                model.fit(X_train, y_train)
            except Exception as e:
                print(f"Erreur entraînement {name}: {e}")

    def create_meta_features(self, X, batch_size=1000):
        import time
        import math
        meta_features = []
        n_samples = X.shape[0]
        for name, model in self.base_models.items():
            parts = []
            for start in range(0, n_samples, batch_size):
                end = min(start + batch_size, n_samples)
                X_batch = X.iloc[start:end] if hasattr(X, 'iloc') else X[start:end]
                probas_batch = model.predict_proba(X_batch)
                parts.append(probas_batch)
            meta_features.append(np.vstack(parts))
        return np.hstack(meta_features)

    def train_meta_model(self, X_train, y_train):
        import xgboost as xgb
        X_meta = self.create_meta_features(X_train)
        self.meta_model = xgb.XGBClassifier(n_estimators=50, max_depth=4, learning_rate=0.1, random_state=49)
        self.meta_model.fit(X_meta, y_train)

    def predict_proba_blended(self, X):
        X_meta = self.create_meta_features(X)
        return self.meta_model.predict_proba(X_meta)

    def predict(self, X, threshold=0.05, min_predictions=5):
        probabilities = self.predict_proba_blended(X)
        if self.per_label_thresholds is None:
            preds = (probabilities > threshold).astype(int)
        else:
            preds = (probabilities > self.per_label_thresholds[np.newaxis, :]).astype(int)

        if min_predictions is not None and min_predictions > 0:
            n_samples, n_labels = probabilities.shape
            for i in range(n_samples):
                if preds[i].sum() >= min_predictions:
                    continue
                sorted_idx = np.argsort(-probabilities[i])
                for idx in sorted_idx:
                    if preds[i, idx] == 0:
                        preds[i, idx] = 1
                    if preds[i].sum() >= min_predictions:
                        break
        return preds, probabilities

# --- Configuration de la page ---
st.set_page_config(page_title="Détection Médicaments & Effets Secondaires", layout="wide")

# --- 1️⃣ Charger le modèle de prédiction d'effets secondaires ---
@st.cache_resource
def load_side_effect_model():
    try:
        model_path = "/content/drive/MyDrive/blender_final_per_label_thresholds (1).pkl"
        saved = joblib.load(model_path)
        blender = saved['predictor']
        return blender
    except Exception as e:
        st.error(f"Erreur chargement modèle effets secondaires: {e}")

        # Créer un modèle de démonstration si le vrai modèle n'est pas disponible
        st.warning("Création d'un modèle de démonstration...")
        demo_model = FixedBlendingPredictor()
        demo_model.top_drugs = ['PARACETAMOL', 'METFORMINE', 'INSULIN', 'ZEPBOUND', 'IBUPROFENE', 'AMOXICILLINE']
        demo_model.top_reactions = [
            'NAUSÉE', 'MAUX DE TÊTE', 'FATIGUE', 'VERTIGES', 'DIARRHÉE',
            'DOULEURS ABDOMINALES', 'ÉRUPTION CUTANÉE', 'SOMMOLENCE', 'ANXIÉTÉ', 'PALPITATIONS'
        ]
        demo_model.feature_names = [f"drug_{drug}" for drug in demo_model.top_drugs] + ['age', 'sex', 'nb_drugs', 'nb_reactions', 'is_sider']
        return demo_model

# --- 2️⃣ Charger les modèles de vision et OCR ---
@st.cache_resource
def load_vision_models():
    try:
        # Charger YOLO
        yolo_model = YOLO("/content/best.pt")

        # Charger TrOCR pour l'écriture manuscrite
        trocr_processor = TrOCRProcessor.from_pretrained("microsoft/trocr-large-handwritten")
        trocr_model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-large-handwritten")

        # Charger le modèle d'embedding pour la similarité sémantique
        sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

        return yolo_model, trocr_processor, trocr_model, sentence_model
    except Exception as e:
        st.error(f"Erreur chargement modèles vision/OCR: {e}")
        st.info("Mode simulation activé pour la détection d'images")
        return None, None, None, None

# --- 3️⃣ Extraction OCR avec TrOCR ---
def extract_text_with_trocr(_processor, _model, image_crop):
    """Extrait le texte d'une image crop avec TrOCR"""
    try:
        # Redimensionner l'image pour de meilleures performances
        if max(image_crop.size) > 512:
            image_crop = image_crop.resize((512, 512), Image.Resampling.LANCZOS)

        # Préparer l'image pour TrOCR
        pixel_values = _processor(images=image_crop, return_tensors="pt").pixel_values

        # Générer la prédiction
        with torch.no_grad():
            generated_ids = _model.generate(pixel_values, max_length=50)

        # Décoder la prédiction
        extracted_text = _processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

        return extracted_text.strip()

    except Exception as e:
        st.error(f"Erreur OCR: {e}")
        return ""

# --- 4️⃣ NOUVELLE FONCTION : Suppression des dosages ---
def remove_dosages(text):
    """
    Détecte et supprime les dosages, concentrations et unités de mesure du texte
    Retourne le texte nettoyé et la liste des éléments supprimés
    """
    if not text:
        return "", []

    original_text = text
    removed_elements = []

    # Pattern pour détecter les dosages avec unités
    dosage_patterns = [
        # Dosages numériques avec unités (500mg, 100g, 50mcg, etc.)
        r'\b\d+\s*(?:mg|mcg|μg|g|kg|ml|l|mL|UI|IU|U\.I\.|µg|mci|ci|mmol|meq|%|m?g/ml|m?g/l)\b',
        # Dosages en toutes lettres (100 milligrammes, 50 grammes, etc.)
        r'\b\d+\s*(?:milligrammes?|grammes?|microgrammes?|litres?|millilitres?|unités?|pourcent|pour\s*cent)\b',
        # Combinaisons avec slash (500mg/50mg, 100mg/5ml, etc.)
        r'\b\d+\s*(?:mg|mcg|g|ml|l)\s*/\s*\d+\s*(?:mg|mcg|g|ml|l)\b',
        # Pourcentages (5%, 10%, etc.)
        r'\b\d+\s*%\b',
        # Unités de mesure générales (10m, 5l, 20cm, etc.)
        r'\b\d+\s*(?:m|cm|mm|km|l|dl|cl|ml|g|dg|cg|mg)\b',
        # Dosages avec points décimaux (12.5mg, 0.5g, etc.)
        r'\b\d+\.\d+\s*(?:mg|mcg|g|ml|l|UI)\b'
    ]

    # Appliquer tous les patterns
    for pattern in dosage_patterns:
        matches = re.findall(pattern, text, re.IGNORECASE)
        for match in matches:
            removed_elements.append(match)
            text = re.sub(re.escape(match), '', text)

    # Nettoyer les espaces multiples et les espaces en début/fin
    text = re.sub(r'\s+', ' ', text).strip()

    # Feedback de débogage
    if removed_elements:
        st.info(f"🔧 Nettoyage dosage: '{original_text}' → Supprimé: {removed_elements} → Résultat: '{text}'")

    return text, removed_elements

# --- 5️⃣ Nettoyage amélioré du texte OCR ---
def enhanced_clean_medication_text(text):
    """Nettoyage amélioré du texte détecté"""
    if not text:
        return ""

    # Étape 1: Supprimer les dosages en premier
    text, removed_dosages = remove_dosages(text)

    # Si après suppression des dosages il ne reste rien, retourner vide
    if not text:
        return ""

    # Étape 2: Convertir en majuscules
    text = text.upper()

    # Étape 3: Supprimer les caractères spéciaux mais garder les lettres, chiffres et espaces
    cleaned = re.sub(r'[^\w\s]', '', text)

    # Étape 4: Corrections OCR améliorées
    corrections = {
        '0': 'O', '1': 'I', '3': 'E', '4': 'A', '5': 'S', '6': 'G',
        '7': 'T', '8': 'B', '9': 'G', 'Ç': 'C', 'É': 'E', 'È': 'E',
        'À': 'A', 'Ù': 'U', 'Ì': 'I', 'Ï': 'I', 'Ü': 'U', 'Ö': 'O',
        'Ñ': 'N', 'Œ': 'OE', 'Æ': 'AE'
    }

    for wrong, right in corrections.items():
        cleaned = cleaned.replace(wrong, right)

    # Étape 5: Corrections de mots courants
    word_corrections = {
        'PARACETAMOL': 'PARACETAMOL',
        'METFORMIN': 'METFORMINE',
        'INSULIN': 'INSULIN',
        'IBUPROFEN': 'IBUPROFENE',
        'AMOXICILLIN': 'AMOXICILLINE',
        'ATORVASTATIN': 'ATORVASTATINE',
        'BISOPROLOL': 'BISOPROLOL',
        'PAROXETIN': 'PAROXETINE',
        'OMEPRAZOLE': 'OMEPRAZOLE',
        'DOLIPRANE': 'PARACETAMOL',
        'EFFERALGAN': 'PARACETAMOL',
        'CLAMOXYL': 'AMOXICILLINE',
        'AUGMENTIN': 'AMOXICILLINE',
        'GLUCOPHAGE': 'METFORMINE'
    }

    # Appliquer les corrections de mots
    for wrong_word, correct_word in word_corrections.items():
        if wrong_word in cleaned:
            cleaned = cleaned.replace(wrong_word, correct_word)

    # Étape 6: Supprimer les espaces multiples et les espaces en début/fin
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()

    return cleaned

# --- 6️⃣ Mapping intelligent des médicaments - VERSION AMÉLIORÉE ---
COMMON_MEDICATIONS = {
    # Analgésiques
    'PARACETAMOL': 'PARACETAMOL', 'DOLIPRANE': 'PARACETAMOL', 'EFFERALGAN': 'PARACETAMOL',
    'ASPIRINE': 'ASPIRINE', 'ASPEGIC': 'ASPIRINE', 'KARDEGIC': 'ASPIRINE',
    'IBUPROFENE': 'IBUPROFENE', 'ADVIL': 'IBUPROFENE', 'NUROFEN': 'IBUPROFENE',

    # Antibiotiques
    'AMOXICILLINE': 'AMOXICILLINE', 'CLAMOXYL': 'AMOXICILLINE', 'AUGMENTIN': 'AMOXICILLINE',
    'PENICILLINE': 'PENICILLINE',

    # Cardiologie
    'ATORVASTATINE': 'ATORVASTATINE', 'TAHOR': 'ATORVASTATINE',
    'BISOPROLOL': 'BISOPROLOL', 'CARDENSIEL': 'BISOPROLOL',

    # Diabète
    'METFORMINE': 'METFORMINE', 'GLUCOPHAGE': 'METFORMINE',
    'INSULINE': 'INSULIN', 'INSULIN': 'INSULIN',

    # Psychiatrie
    'PAROXETINE': 'PAROXETINE', 'DEROXAT': 'PAROXETINE',

    # Gastro
    'OMEPRAZOLE': 'OMEPRAZOLE', 'MOPRAL': 'OMEPRAZOLE',

    # Exemples fournis
    'ZEPBOUND': 'ZEPBOUND', 'METFORMIN': 'METFORMINE', 'INSULIN': 'INSULIN'
}

def find_levenshtein_match(detected_text, known_medications, max_distance_ratio=0.3):
    """Trouve le meilleur match basé sur la distance de Levenshtein"""
    try:
        from Levenshtein import distance, ratio

        best_match = None
        best_ratio = 0

        for known_med in known_medications:
            # Calculer la similarité
            similarity_ratio = ratio(detected_text, known_med)

            # Prendre le ratio de similarité comme métrique principale
            if similarity_ratio > best_ratio:
                best_ratio = similarity_ratio
                best_match = known_med

        # Appliquer un seuil de similarité
        if best_ratio >= (1 - max_distance_ratio):
            st.info(f"Match Levenshtein: '{detected_text}' → '{best_match}' (similarité: {best_ratio:.2f})")
            return best_match

        return None

    except ImportError:
        st.warning("Levenshtein non installé, utilisation de la similarité simple")
        return find_simple_string_match(detected_text, known_medications)
    except Exception as e:
        st.error(f"Erreur Levenshtein: {e}")
        return find_simple_string_match(detected_text, known_medications)

def find_simple_string_match(detected_text, known_medications, threshold=0.7):
    """Matching simple basé sur les caractères communs si Levenshtein n'est pas disponible"""
    best_match = None
    best_similarity = 0

    for known_med in known_medications:
        # Similarité basée sur les caractères communs
        set1, set2 = set(detected_text.upper()), set(known_med.upper())
        common_chars = set1 & set2
        similarity = len(common_chars) / max(len(set1), len(set2)) if max(len(set1), len(set2)) > 0 else 0

        if similarity > best_similarity:
            best_similarity = similarity
            best_match = known_med

    if best_similarity >= threshold:
        st.info(f"Match simple: '{detected_text}' → '{best_match}' (similarité: {best_similarity:.2f})")
        return best_match

    return None

def find_semantic_match(detected_text, known_medications, sentence_model, threshold=0.6):
    """Trouve le meilleur match sémantique"""
    try:
        # Créer les embeddings
        detected_embedding = sentence_model.encode([detected_text])
        known_embeddings = sentence_model.encode(list(known_medications))

        # Calculer les similarités cosinus
        similarities = cosine_similarity(detected_embedding, known_embeddings)[0]

        # Trouver le meilleur match
        best_idx = np.argmax(similarities)
        best_similarity = similarities[best_idx]
        best_match = list(known_medications.keys())[best_idx]

        if best_similarity >= threshold:
            st.info(f"Match sémantique: '{detected_text}' → '{best_match}' (confiance: {best_similarity:.2f})")
            return best_match
        else:
            st.warning(f"Similarité faible ({best_similarity:.2f}) pour '{detected_text}' → '{best_match}'")
            return best_match  # Retourner quand même le meilleur match même si faible

    except Exception as e:
        st.error(f"Erreur matching sémantique: {e}")
        return detected_text

def find_best_match(detected_text, known_medications, sentence_model, threshold=0.6):
    """Trouve le meilleur match avec maximisation de la ressemblance caractère par caractère"""
    try:
        if not detected_text:
            return None

        # Nettoyer le texte détecté
        cleaned_text = enhanced_clean_medication_text(detected_text)

        # Si le texte nettoyé est vide
        if not cleaned_text:
            return None

        # 1. Vérifier d'abord les correspondances exactes
        for known_med in known_medications:
            if known_med == cleaned_text:
                return known_med

        # 2. Correspondances partielles (un médicament contient l'autre ou vice versa)
        for known_med in known_medications:
            if known_med in cleaned_text or cleaned_text in known_med:
                st.info(f"Correspondance partielle: '{cleaned_text}' → '{known_med}'")
                return known_med

        # 3. Similarité de Levenshtein (distance d'édition) pour ressemblance caractère par caractère
        best_levenshtein_match = find_levenshtein_match(cleaned_text, known_medications)
        if best_levenshtein_match:
            return best_levenshtein_match

        # 4. Similarité sémantique avec Sentence-BERT (fallback)
        if sentence_model is not None:
            return find_semantic_match(cleaned_text, known_medications, sentence_model, threshold)

        # 5. Si rien ne marche, retourner le texte nettoyé
        return cleaned_text

    except Exception as e:
        st.error(f"Erreur dans le matching: {e}")
        return enhanced_clean_medication_text(detected_text)

def map_to_standard_medication(detected_text, sentence_model):
    """Mappe le texte détecté vers des noms standardisés de médicaments"""
    return find_best_match(detected_text, COMMON_MEDICATIONS, sentence_model)

# --- 7️⃣ Détection et reconnaissance complète ---
def detect_and_recognize_medications(image, yolo_model, trocr_processor, trocr_model, sentence_model):
    """Détecte les médicaments avec YOLO et extrait le texte avec TrOCR"""
    detected_medications = []

    try:
        # Détection YOLO
        results = yolo_model.predict(source=image, conf=0.5, save=False)

        # Image pour dessiner les bounding boxes
        draw_img = image.copy()
        draw = ImageDraw.Draw(draw_img)

        for r in results:
            boxes = r.boxes.xyxy.cpu().numpy()
            confidences = r.boxes.conf.cpu().numpy()

            for i, (box, conf) in enumerate(zip(boxes, confidences)):
                x1, y1, x2, y2 = map(int, box)

                # Dessiner la bounding box
                draw.rectangle([x1, y1, x2, y2], outline="red", width=3)
                draw.text((x1, y1-10), f"Med {i+1} ({conf:.2f})", fill="red")

                # Extraire la région d'intérêt (ROI)
                roi = image.crop((x1, y1, x2, y2))

                # OCR avec TrOCR
                if trocr_processor and trocr_model:
                    extracted_text = extract_text_with_trocr(trocr_processor, trocr_model, roi)
                else:
                    # Fallback: simulation
                    extracted_text = f"MEDICAMENT_{i+1}"

                if extracted_text:
                    # Afficher le texte OCR brut détecté
                    st.info(f"📝 Texte OCR brut détecté: '{extracted_text}'")

                    # Mapping vers un médicament standard
                    mapped_med = map_to_standard_medication(extracted_text, sentence_model)

                    if mapped_med:
                        detected_medications.append({
                            'original_text': extracted_text,
                            'mapped_med': mapped_med,
                            'confidence': conf,
                            'bbox': (x1, y1, x2, y2)
                        })

        return detected_medications, draw_img

    except Exception as e:
        st.error(f"Erreur lors de la détection: {e}")
        # Simulation de fallback
        simulated_meds = [
            {'original_text': 'PARACETAMOL', 'mapped_med': 'PARACETAMOL', 'confidence': 0.95, 'bbox': (50, 50, 200, 100)},
            {'original_text': 'METFORMINE', 'mapped_med': 'METFORMINE', 'confidence': 0.92, 'bbox': (50, 150, 200, 200)}
        ]
        return simulated_meds, image

# --- 8️⃣ Prédiction des effets secondaires ---
def predict_side_effects(medications, age, sex, blender_model, min_predictions=5):
    """Prédit les effets secondaires basés sur les médicaments"""
    try:
        # Préparation des features
        patient_features = {}

        # Encodage des médicaments
        drugs_up = [med.upper() for med in medications]
        for drug in blender_model.top_drugs:
            patient_features[f"drug_{drug}"] = 1 if drug in drugs_up else 0

        # Autres features
        patient_features["age"] = age
        patient_features["sex"] = sex
        patient_features["nb_drugs"] = len(medications)
        patient_features["nb_reactions"] = 0
        patient_features["is_sider"] = 0

        # Création du DataFrame
        patient_df = pd.DataFrame([patient_features])

        # Assurer toutes les colonnes nécessaires
        for col in blender_model.feature_names:
            if col not in patient_df.columns:
                patient_df[col] = 0
        patient_df = patient_df[blender_model.feature_names]

        # Prédiction (simulation si modèle de démo)
        if hasattr(blender_model, 'predict'):
            preds, probas = blender_model.predict(patient_df, min_predictions=min_predictions)
        else:
            # Simulation pour le modèle de démonstration
            np.random.seed(hash(str(medications)) % 10000)
            probas = np.random.uniform(0, 0.3, (1, len(blender_model.top_reactions)))
            preds = (probas > 0.15).astype(int)

            # Assurer le nombre minimum de prédictions
            if preds.sum() < min_predictions:
                sorted_idx = np.argsort(-probas[0])
                for idx in sorted_idx:
                    if preds[0, idx] == 0:
                        preds[0, idx] = 1
                    if preds.sum() >= min_predictions:
                        break

        # Formatage des résultats
        results_list = []
        for i, reaction in enumerate(blender_model.top_reactions):
            if preds[0, i] == 1:
                prob = probas[0][i]
                confidence = '🔴 HAUTE' if prob > 0.7 else '🟡 MOYENNE' if prob > 0.4 else '🟢 FAIBLE'
                results_list.append({
                    'reaction': reaction,
                    'probability': float(prob),
                    'confidence': confidence
                })

        results_list.sort(key=lambda x: x['probability'], reverse=True)
        return results_list, patient_df

    except Exception as e:
        st.error(f"Erreur lors de la prédiction: {e}")
        return [], None

# --- 9️⃣ Analyse SHAP ---
def analyze_with_shap(blender_model, patient_df, selected_reaction):
    """Analyse SHAP pour expliquer la prédiction d'une réaction spécifique"""
    try:
        # Vérifier que le modèle a les attributs nécessaires
        if not hasattr(blender_model, 'base_models') or 'rf1' not in blender_model.base_models:
            st.warning("Modèle incompatible pour l'analyse SHAP")
            return None

        # Trouver l'index de la réaction
        if selected_reaction not in blender_model.top_reactions:
            st.warning(f"Réaction '{selected_reaction}' non trouvée dans le modèle")
            return None

        reaction_idx = blender_model.top_reactions.index(selected_reaction)

        # Obtenir le classifieur Random Forest pour cette réaction
        rf_classifier = blender_model.base_models['rf1'].estimators_[reaction_idx]

        # Créer l'explainer SHAP
        explainer = shap.TreeExplainer(rf_classifier)

        # Calculer les valeurs SHAP
        shap_values = explainer.shap_values(patient_df)

        # Gérer différents formats de sortie SHAP
        if isinstance(shap_values, list):
            # Cas multi-classes: prendre les valeurs pour la classe positive
            if len(shap_values) == 2:
                shap_vals = shap_values[1]  # Classe positive
            else:
                shap_vals = shap_values[0]
        else:
            shap_vals = shap_values

        # S'assurer que c'est un array 1D
        if len(shap_vals.shape) > 1:
            shap_vals = shap_vals.flatten()

        return {
            'explainer': explainer,
            'shap_values': shap_vals,
            'expected_value': explainer.expected_value[1] if isinstance(explainer.expected_value, (list, np.ndarray)) else explainer.expected_value
        }

    except Exception as e:
        st.error(f"Erreur lors de l'analyse SHAP: {e}")
        return None

def plot_shap_analysis(shap_results, patient_df, feature_names):
    """Crée un graphique SHAP pour visualiser les contributions"""
    try:
        fig, ax = plt.subplots(figsize=(10, 6))

        # Créer un DataFrame pour les features et leurs contributions
        contributions = []
        for i, feature in enumerate(feature_names):
            contributions.append({
                'feature': feature,
                'contribution': shap_results['shap_values'][i],
                'value': patient_df[feature].iloc[0]
            })

        contrib_df = pd.DataFrame(contributions)

        # Séparer les contributions positives et négatives
        positive_contrib = contrib_df[contrib_df['contribution'] > 0].nlargest(5, 'contribution')
        negative_contrib = contrib_df[contrib_df['contribution'] < 0].nsmallest(5, 'contribution')

        # Préparer les données pour le graphique
        features_plot = []
        values_plot = []
        colors_plot = []

        for _, row in positive_contrib.iterrows():
            features_plot.append(f"{row['feature']}\n(valeur: {row['value']})")
            values_plot.append(row['contribution'])
            colors_plot.append('red')

        for _, row in negative_contrib.iterrows():
            features_plot.append(f"{row['feature']}\n(valeur: {row['value']})")
            values_plot.append(row['contribution'])
            colors_plot.append('blue')

        # Créer le graphique à barres
        y_pos = np.arange(len(features_plot))
        bars = ax.barh(y_pos, values_plot, color=colors_plot, alpha=0.7)

        # Ajouter les étiquettes
        ax.set_yticks(y_pos)
        ax.set_yticklabels(features_plot, fontsize=10)
        ax.set_xlabel('Contribution SHAP', fontsize=12)
        ax.set_title('Facteurs influençant la prédiction', fontsize=14, fontweight='bold')

        # Ajouter une légende
        ax.axvline(x=0, color='black', linestyle='-', alpha=0.3)

        # Améliorer le layout
        plt.tight_layout()

        return fig

    except Exception as e:
        st.error(f"Erreur lors de la création du graphique SHAP: {e}")
        return None

def create_shap_summary(shap_results, patient_df, selected_reaction):
    """Crée un résumé textuel de l'analyse SHAP"""
    try:
        if shap_results is None:
            return "Analyse SHAP non disponible"

        contributions = []
        for i, feature in enumerate(patient_df.columns):
            contribution = shap_results['shap_values'][i]
            value = patient_df[feature].iloc[0]

            # Formater le nom de la feature
            if feature.startswith('drug_'):
                drug_name = feature.replace('drug_', '')
                status = "PRÉSENT" if value == 1 else "ABSENT"
                display_name = f"Médicament {drug_name} ({status})"
            elif feature == "age":
                display_name = f"Âge ({int(value)} ans)"
            elif feature == "sex":
                gender = "Homme" if value == 1 else "Femme"
                display_name = f"Sexe ({gender})"
            elif feature == "nb_drugs":
                display_name = f"Nombre de médicaments ({int(value)})"
            elif feature == "nb_reactions":
                display_name = f"Nombre de réactions ({int(value)})"
            else:
                display_name = feature

            contributions.append({
                'feature': display_name,
                'contribution': contribution,
                'value': value
            })

        contrib_df = pd.DataFrame(contributions)

        # Générer le résumé
        summary = f"## 🔍 Analyse SHAP pour: {selected_reaction}\n\n"

        # Facteurs qui augmentent le risque
        positive_factors = contrib_df[contrib_df['contribution'] > 0.001].nlargest(3, 'contribution')
        if len(positive_factors) > 0:
            summary += "### ➕ Facteurs augmentant le risque:\n"
            for _, factor in positive_factors.iterrows():
                summary += f"- **{factor['feature']}**: +{factor['contribution']:.3f}\n"

        # Facteurs qui réduisent le risque
        negative_factors = contrib_df[contrib_df['contribution'] < -0.001].nsmallest(3, 'contribution')
        if len(negative_factors) > 0:
            summary += "\n### ➖ Facteurs réduisant le risque:\n"
            for _, factor in negative_factors.iterrows():
                summary += f"- **{factor['feature']}**: {factor['contribution']:.3f}\n"

        # Probabilité de base
        if 'expected_value' in shap_results:
            summary += f"\n### 📊 Métriques:\n"
            summary += f"- **Probabilité de base**: {shap_results['expected_value']:.3f}\n"
            summary += f"- **Impact total**: {sum(shap_results['shap_values']):.3f}\n"

        return summary

    except Exception as e:
        return f"Erreur lors de la génération du résumé SHAP: {e}"

# --- 🔟 Exemples prédéfinis ---
PREDEFINED_EXAMPLES = {
    "Diabète (Metformine + Insuline)": {
        "medications": ["METFORMINE", "INSULIN"],
        "age": 52,
        "sex": 1,
        "description": "Patient diabétique sous insulinothérapie"
    },
    "Obésité (Zepbound)": {
        "medications": ["ZEPBOUND"],
        "age": 42,
        "sex": 2,
        "description": "Traitement pour perte de poids"
    },
    "Douleurs (Paracétamol)": {
        "medications": ["PARACETAMOL"],
        "age": 35,
        "sex": 1,
        "description": "Antalgique courant"
    },
    "Hypertension (Bisoprolol + Atorvastatine)": {
        "medications": ["BISOPROLOL", "ATORVASTATINE"],
        "age": 65,
        "sex": 1,
        "description": "Traitement cardiovasculaire"
    },
    "Infection (Amoxicilline)": {
        "medications": ["AMOXICILLINE"],
        "age": 28,
        "sex": 2,
        "description": "Antibiotique pour infection"
    }
}

# --- INTERFACE STREAMLIT ---
st.title("💊 Détection de Médicaments & Prédiction d'Effets Secondaires")

# Charger les modèles
blender_model = load_side_effect_model()
yolo_model, trocr_processor, trocr_model, sentence_model = load_vision_models()

# Sidebar pour les paramètres
st.sidebar.header("Paramètres du Patient")

age = st.sidebar.number_input("Âge", min_value=1, max_value=120, value=45)
sex = st.sidebar.selectbox("Sexe", options=[1, 2], format_func=lambda x: "Homme" if x == 1 else "Femme")
min_predictions = st.sidebar.slider("Nombre minimum d'effets à prédire", 1, 15, 5)

# Onglets principaux
tab1, tab2, tab3 = st.tabs(["📷 Analyse d'Image", "💊 Saisie Manuelle", "📋 Exemples Prédéfinis"])

with tab1:
    st.header("Analyse d'Image par Caméra/Upload")

    uploaded_file = st.file_uploader("Choisir une image de médicaments...",
                                   type=["jpg", "jpeg", "png"])

    col1, col2 = st.columns(2)

    detected_medications = []

    with col1:
        if uploaded_file:
            image = Image.open(uploaded_file).convert("RGB")
            st.image(image, caption="Image uploadée", use_container_width=True)

            if st.button("🔍 Analyser l'Image", type="primary"):
                with st.spinner("Détection et reconnaissance des médicaments en cours..."):
                    if yolo_model is not None:
                        # Détection et reconnaissance complète
                        detected_results, annotated_image = detect_and_recognize_medications(
                            image, yolo_model, trocr_processor, trocr_model, sentence_model
                        )

                        # Afficher l'image annotée
                        st.image(annotated_image, caption="Médicaments détectés", use_container_width=True)

                        # Afficher les résultats de détection
                        if detected_results:
                            st.subheader("📋 Résultats de la Reconnaissance")

                            for i, detection in enumerate(detected_results, 1):
                                col_a, col_b, col_c = st.columns([2, 1, 1])
                                with col_a:
                                    st.write(f"**{i}. {detection['mapped_med']}**")
                                with col_b:
                                    st.write(f"Conf: {detection['confidence']:.2f}")
                                with col_c:
                                    if detection['original_text'] != detection['mapped_med']:
                                        st.info(f"OCR: '{detection['original_text']}'")

                            # Stocker les médicaments mappés pour la prédiction
                            detected_medications = [det['mapped_med'] for det in detected_results]
                            st.session_state.detected_meds = detected_medications
                            st.session_state.detection_details = detected_results

                        else:
                            st.warning("Aucun médicament détecté dans l'image.")
                            st.session_state.detected_meds = []

                    else:
                        # Mode simulation
                        st.info("Mode simulation - Détection simulée")
                        simulated_meds = ["PARACETAMOL", "METFORMINE"]
                        st.session_state.detected_meds = simulated_meds
                        for med in simulated_meds:
                            st.success(f"✅ {med}")

    with col2:
        if 'detected_meds' in st.session_state and st.session_state.detected_meds:
            st.subheader("🔮 Effets Secondaires Prédits")

            results, patient_df = predict_side_effects(
                st.session_state.detected_meds,
                age, sex, blender_model, min_predictions
            )

            if results:
                st.metric("Nombre d'effets détectés", len(results))

                # Afficher les prédictions
                for i, pred in enumerate(results[:10], 1):
                    with st.container():
                        col_a, col_b = st.columns([3, 1])
                        with col_a:
                            st.write(f"**{i}. {pred['reaction']}**")
                        with col_b:
                            st.write(f"{pred['probability']:.1%} {pred['confidence']}")
                        st.progress(pred['probability'])

                # Section SHAP
                st.subheader("🔍 Analyse SHAP")
                if len(results) > 0:
                    selected_reaction = st.selectbox(
                        "Choisir une réaction à analyser:",
                        options=[r['reaction'] for r in results[:5]],
                        key="shap_selection_tab1"
                    )

                    if selected_reaction and patient_df is not None:
                        with st.spinner("Analyse SHAP en cours..."):
                            shap_results = analyze_with_shap(blender_model, patient_df, selected_reaction)

                            if shap_results is not None:
                                # Afficher le résumé textuel
                                shap_summary = create_shap_summary(shap_results, patient_df, selected_reaction)
                                st.markdown(shap_summary)

                                # Afficher le graphique
                                shap_plot = plot_shap_analysis(shap_results, patient_df, patient_df.columns.tolist())
                                if shap_plot is not None:
                                    st.pyplot(shap_plot)
            else:
                st.info("Aucun effet secondaire significatif prédit.")

with tab2:
    st.header("Saisie Manuelle des Médicaments")

    manual_meds = st.text_area(
        "Entrez les noms des médicaments (un par ligne)",
        placeholder="METFORMINE\nINSULIN\nPARACETAMOL",
        height=100
    )

    if st.button("Prédire les Effets Secondaires", key="manual_predict"):
        if manual_meds:
            medications = [med.strip().upper() for med in manual_meds.split('\n') if med.strip()]

            # Nettoyage et mapping
            cleaned_meds = []
            for med in medications:
                mapped = map_to_standard_medication(med, sentence_model)
                cleaned_meds.append(mapped)
                if mapped != med:
                    st.info(f"'{med}' → '{mapped}'")

            st.write("**Médicaments analysés:**", ", ".join(cleaned_meds))

            results, patient_df = predict_side_effects(cleaned_meds, age, sex, blender_model, min_predictions)

            if results:
                st.subheader("🔮 Effets Secondaires Prédits")
                st.metric("Nombre d'effets détectés", len(results))

                # Afficher les prédictions
                for i, pred in enumerate(results[:10], 1):
                    with st.container():
                        col_a, col_b = st.columns([3, 1])
                        with col_a:
                            st.write(f"**{i}. {pred['reaction']}**")
                        with col_b:
                            st.write(f"{pred['probability']:.1%} {pred['confidence']}")
                        st.progress(pred['probability'])

                # Section SHAP
                st.subheader("🔍 Analyse SHAP")
                if len(results) > 0:
                    selected_reaction = st.selectbox(
                        "Choisir une réaction à analyser:",
                        options=[r['reaction'] for r in results[:5]],
                        key="shap_selection_tab2"
                    )

                    if selected_reaction and patient_df is not None:
                        with st.spinner("Analyse SHAP en cours..."):
                            shap_results = analyze_with_shap(blender_model, patient_df, selected_reaction)

                            if shap_results is not None:
                                # Afficher le résumé textuel
                                shap_summary = create_shap_summary(shap_results, patient_df, selected_reaction)
                                st.markdown(shap_summary)

                                # Afficher le graphique
                                shap_plot = plot_shap_analysis(shap_results, patient_df, patient_df.columns.tolist())
                                if shap_plot is not None:
                                    st.pyplot(shap_plot)
            else:
                st.warning("Aucun effet secondaire significatif prédit pour cette combinaison.")

with tab3:
    st.header("Exemples Prédéfinis")

    selected_example = st.selectbox("Choisir un exemple", list(PREDEFINED_EXAMPLES.keys()))

    if selected_example:
        example_data = PREDEFINED_EXAMPLES[selected_example]

        st.write(f"**Description:** {example_data['description']}")
        st.write(f"**Médicaments:** {', '.join(example_data['medications'])}")
        st.write(f"**Âge:** {example_data['age']} ans")
        st.write(f"**Sexe:** {'Homme' if example_data['sex'] == 1 else 'Femme'}")

        if st.button("Lancer la Prédiction", key="example_predict"):
            results, patient_df = predict_side_effects(
                example_data['medications'],
                example_data['age'],
                example_data['sex'],
                blender_model,
                min_predictions
            )

            if results:
                st.subheader("📊 Résultats de Prédiction")
                st.metric("Nombre d'effets détectés", len(results))

                # Afficher les prédictions
                for i, pred in enumerate(results[:10], 1):
                    with st.container():
                        col_a, col_b = st.columns([3, 1])
                        with col_a:
                            st.write(f"**{i}. {pred['reaction']}**")
                        with col_b:
                            st.write(f"{pred['probability']:.1%} {pred['confidence']}")
                        st.progress(pred['probability'])

                # Section SHAP
                st.subheader("🔍 Analyse SHAP")
                if len(results) > 0:
                    selected_reaction = st.selectbox(
                        "Choisir une réaction à analyser:",
                        options=[r['reaction'] for r in results[:5]],
                        key="shap_selection_tab3"
                    )

                    if selected_reaction and patient_df is not None:
                        with st.spinner("Analyse SHAP en cours..."):
                            shap_results = analyze_with_shap(blender_model, patient_df, selected_reaction)

                            if shap_results is not None:
                                # Afficher le résumé textuel
                                shap_summary = create_shap_summary(shap_results, patient_df, selected_reaction)
                                st.markdown(shap_summary)

                                # Afficher le graphique
                                shap_plot = plot_shap_analysis(shap_results, patient_df, patient_df.columns.tolist())
                                if shap_plot is not None:
                                    st.pyplot(shap_plot)

                # Graphique des probabilités
                if len(results) > 0:
                    chart_data = pd.DataFrame({
                        'Effets Secondaires': [pred['reaction'] for pred in results[:8]],
                        'Probabilité': [pred['probability'] for pred in results[:8]]
                    })
                    st.bar_chart(chart_data, x='Effets Secondaires', y='Probabilité')

# Footer
st.markdown("---")
st.markdown("*Système de prédiction des effets secondaires - À des fins éducatives seulement*")

# Information de débogage
with st.sidebar:
    st.markdown("---")
    st.markdown("**Informations techniques:**")
    st.write(f"Médicaments connus: {len(blender_model.top_drugs)}")
    st.write(f"Effets secondaires: {len(blender_model.top_reactions)}")
    st.write(f"YOLO chargé: {yolo_model is not None}")
    st.write(f"TrOCR chargé: {trocr_model is not None}")
    st.write(f"Embeddings chargés: {sentence_model is not None}")

Overwriting streamlit_app.py


In [6]:
# Installation des dépendances
!pip install --quiet streamlit ultralytics transformers torch pillow pyngrok scikit-learn==1.7.2 xgboost

# Vérifier que le fichier modèle est présent
import os
if not os.path.exists("/content/drive/MyDrive/blender_final_per_label_thresholds (1).pkl"):
    print("⚠️  Fichier modèle non trouvé, l'application fonctionnera en mode démonstration")

# Lancer Streamlit
from pyngrok import ngrok
import threading
import subprocess
import time

def run_streamlit():
    subprocess.run(["streamlit", "run", "streamlit_app.py", "--server.port", "8501", "--server.address", "0.0.0.0"])

# Démarrer dans un thread
thread = threading.Thread(target=run_streamlit)
thread.daemon = True
thread.start()

time.sleep(8)  # Attendre que Streamlit démarre

# Créer le tunnel ngrok
public_url = ngrok.connect(8501)
print(f"🎯 Application accessible à: {public_url}")

🎯 Application accessible à: NgrokTunnel: "https://ecdba8a7de4b.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!pkill -f ngrok && fuser -k 8501/tcp && echo "✅ Nettoyage terminé"


^C
